In [ ]:
# !pip install transformers
# !pip install datasets

In [14]:
import pandas as pd
import torch
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments
from transformers import Trainer
import warnings
from transformers import BertTokenizer, BertModel
warnings.filterwarnings("ignore")

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [16]:
train_df=pd.read_csv('EmbibleDataset/train_df.csv', sep='\\t', encoding='utf-8')
valid_df=pd.read_csv('EmbibleDataset/valid_df.csv', sep='\\t', encoding='utf-8')

In [17]:
train_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
train_df.rename(columns={"name.1": 'name'}, inplace=True)
#valid_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
#valid_df.rename(columns={"name.1": 'name'}, inplace=True)

In [18]:
print(train_df)
print(valid_df)

      verse_idx   verse                                               name
0        Isaiah     0.0  חֲזוֹן֙ יְשַֽׁעְיָ֣הוּ בֶן אָמ֔וֹץ אֲשֶׁ֣ר חָז...
1        Isaiah     1.0  שִׁמְע֤וּ שָׁמַ֙יִם֙ וְהַאֲזִ֣ינִי אֶ֔רֶץ כִּ֥...
2        Isaiah     2.0  יָדַ֥ע שׁוֹר֙ קֹנֵ֔הוּ וַחֲמ֖וֹר אֵב֣וּס בְּעָ...
3        Isaiah     3.0  ה֣וֹי גּ֣וֹי חֹטֵ֗א עַ֚ם כֶּ֣בֶד עָוֺ֔ן זֶ֣רַע...
4        Isaiah     4.0  עַ֣ל מֶ֥ה תֻכּ֛וּ ע֖וֹד תּוֹסִ֣יפוּ סָרָ֑ה כָּ...
...         ...     ...                                                ...
22145    Exodus  1208.0  וַיְכַ֥ס הֶעָנָ֖ן אֶת אֹ֣הֶל מוֹעֵ֑ד וּכְב֣וֹד...
22146    Exodus  1209.0  וְלֹא יָכֹ֣ל מֹשֶׁ֗ה לָבוֹא֙ אֶל אֹ֣הֶל מוֹעֵ֔...
22147    Exodus  1210.0  וּבְהֵעָל֤וֹת הֶֽעָנָן֙ מֵעַ֣ל הַמִּשְׁכָּ֔ן י...
22148    Exodus  1211.0  וְאִם לֹ֥א יֵעָלֶ֖ה הֶעָנָ֑ן וְלֹ֣א יִסְע֔וּ ע...
22149    Exodus  1212.0  כִּי֩ עֲנַ֨ן יְהוָ֤ה עַֽל הַמִּשְׁכָּן֙ יוֹמָ֔...

[22150 rows x 3 columns]
         name  verse_idx                                              vers

In [19]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# model = BertModel.from_pretrained("bert-base-multilingual-uncased")

# cp = "tau/tavbert-he"
model=AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-uncased")
tokenizer=AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

In [20]:
train_ds=datasets.Dataset.from_pandas(train_df)
valid_ds=datasets.Dataset.from_pandas(valid_df)

In [21]:

def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"]))
tokenized_train_ds=train_ds.map(tokenize_function)
tokenized_valid_ds=valid_ds.map(tokenize_function)


  0%|          | 0/22150 [00:00<?, ?ex/s]

  0%|          | 0/536 [00:00<?, ?ex/s]

In [22]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [23]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-NormalBert-10epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    num_train_epochs=10
)
#training_args = TrainingArguments("test-trainer",num_train_epochs=10)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
# from sklearn.metrics import f1_score

# def compute_metrics(pred):
#   return {'f1:':pred}

In [25]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse, name, verse_idx. If verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3470
  Number of trainable parameters = 167463831
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.150000,5.984001
2,2.119500,6.764743
3,2.135600,8.432836
4,2.057600,8.031181
5,2.158200,8.931014
6,2.138600,9.223641
7,2.128300,8.655166
8,2.075500,9.058912
9,2.107500,8.816860
10,2.136200,9.040538


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__, verse, name, verse_idx. If __index_level_0__, verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-10epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argume

TrainOutput(global_step=3470, training_loss=2.120715068190517, metrics={'train_runtime': 672.5892, 'train_samples_per_second': 329.324, 'train_steps_per_second': 5.159, 'total_flos': 682822798042032.0, 'train_loss': 2.120715068190517, 'epoch': 10.0})

before training 10.36 after 9.37

In [27]:
trainer.save_model("EmbibleModels/Embibert-finetuned-NormalBert-10epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-NormalBert-10epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-10epochs
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-10epochs/pytorch_model.bin


In [28]:
model=AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-uncased")
tokenizer=AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

In [29]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-NormalBert-20epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    num_train_epochs=20
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [30]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse, name, verse_idx. If verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6940
  Number of trainable parameters = 167463831


Epoch,Training Loss,Validation Loss
1,2.150500,6.051132
2,2.118300,7.462174
3,2.136600,8.595393
4,2.059900,8.554097
5,2.159100,10.103720
6,2.144900,10.146825
7,2.133200,10.348342
8,2.080300,10.605648
9,2.113700,10.596652
10,2.140400,11.716318


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__, verse, name, verse_idx. If __index_level_0__, verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-20epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argume

TrainOutput(global_step=6940, training_loss=2.108155325853859, metrics={'train_runtime': 1384.0366, 'train_samples_per_second': 320.078, 'train_steps_per_second': 5.014, 'total_flos': 1365385246918812.0, 'train_loss': 2.108155325853859, 'epoch': 20.0})

In [31]:
trainer.save_model("EmbibleModels/Embibert-finetuned-NormalBert-20epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-NormalBert-20epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-20epochs
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-20epochs/pytorch_model.bin


In [32]:
model=AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-uncased")
tokenizer=AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

In [33]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-NormalBert-50epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    num_train_epochs=50
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse, name, verse_idx. If verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 17350
  Number of trainable parameters = 167463831


Epoch,Training Loss,Validation Loss
1,2.150500,6.225634
2,2.118800,7.624661
3,2.140200,8.652073
4,2.061900,8.300903
5,2.161500,9.615662
6,2.145200,9.923818
7,2.141900,10.786325
8,2.084600,10.838971
9,2.116000,11.285677
10,2.144400,12.821379


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__, verse, name, verse_idx. If __index_level_0__, verse, name, verse_idx are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 64
Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-50epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argume

TrainOutput(global_step=17350, training_loss=2.0958767533096183, metrics={'train_runtime': 4163.9206, 'train_samples_per_second': 265.975, 'train_steps_per_second': 4.167, 'total_flos': 3413345291291412.0, 'train_loss': 2.0958767533096183, 'epoch': 50.0})

In [35]:
trainer.save_model("EmbibleModels/Embibert-finetuned-NormalBert-50epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-NormalBert-50epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-NormalBert-50epochs
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-NormalBert-50epochs/pytorch_model.bin
